In [120]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import requests
from requests.exceptions import HTTPError
import json
from typing import List, Set

class Question:
    def __init__(self, id: int, titleSlug: str, url: str=''):
        self.id = id
        self.titleSlug = titleSlug
        self.url = f"https://leetcode.com/problems/{titleSlug}/"
    
    def __str__(self):
        return f"[{self.id}. {self.titleSlug}]"
    
    def __repr__(self):
        return f"{self.id}: {self.url}"
    
    def __hash__(self):
        return self.id
    
    def __eq__(self, other):
        return isinstance(other, Question) and other.id == self.id and other.titleSlug == self.titleSlug

class LeetcodeClient:
    _transport = RequestsHTTPTransport(
        url='https://leetcode.com/graphql',
        use_json=True,
    )
    
    _client = Client(
        transport=_transport,
        fetch_schema_from_transport=True,
    )
    
    _questionCache = {}
    
    def getSimilarQuestions(self, title: str) -> List[Question]:
        query = gql("""
            query getQuestionDetail($titleSlug: String!) {
              question(titleSlug: $titleSlug) {
                similarQuestions
              }
            }
            """)
        params = {"titleSlug": title}
        json_dict = LeetcodeClient._client.execute(query, variable_values=params)
        if not json_dict:
            return []
        similar_qstr = json_dict['question']['similarQuestions']
        similar_qarr = json.loads(similar_qstr)
        return [Question(self.getQuestionId(q['titleSlug']), q['titleSlug']) for q in similar_qarr]

    def toTitle(self, questions: List[Question]) -> List[str]:
        return map(lambda x: x.titleSlug, curset)
    
    def getSimilarQuestionCluster(self, question: Question) -> List[Question]:
        q = self.getSimilarQuestions(question.titleSlug)
        visited = set(q+[question])
        depth = 3
        h = 0
        while q and h < depth:
            l = len(q)
            h += 1
            for _ in range(l):
                cur_title = q.pop(0).titleSlug
                new_q = self.getSimilarQuestions(cur_title)
                new_questions = list(set(new_q) - visited)
                new_qset = list(map(lambda x: x.titleSlug, new_questions))
#                 print(f"{cur_title} -> {new_qset}")
                q.extend(new_questions)
                visited |= set(new_q)
        return sorted(list(visited), key=lambda x: x.id)
    
    def getlAllQuestions(self) -> List[Question]:
        url = 'https://leetcode.com/api/problems/all'
        try:
            response = requests.get(url)
            # If the response was successful, no Exception will be raised
            response.raise_for_status()
            result = response.json()
            questions = result['stat_status_pairs']
            res = []
            for ele in questions:
                q = Question(ele['stat']['question_id'], ele['stat']['question__title_slug'])
                LeetcodeClient._questionCache[q.titleSlug] = q
                res.append(q)
            return sorted(res, key=lambda q: q.id)
        except HTTPError as http_err:
            print(f'HTTP error occurred: {http_err}')  # Python 3.6
        except Exception as err:
            print(f'Other error occurred: {err}')  # Python 3.6
        else:
            print('Success!')
        return []
    
    def getQuestionId(self, title: str) -> int:
        default = Question(0, '')
        return LeetcodeClient._questionCache.get(title, default).id

In [128]:
lc_client = LeetcodeClient()
questions = lc_client.getlAllQuestions()

In [129]:
title = 'longest-palindromic-substring'
lc_client.getSimilarQuestions(title)

[214: https://leetcode.com/problems/shortest-palindrome/,
 266: https://leetcode.com/problems/palindrome-permutation/,
 336: https://leetcode.com/problems/palindrome-pairs/,
 516: https://leetcode.com/problems/longest-palindromic-subsequence/,
 647: https://leetcode.com/problems/palindromic-substrings/]

In [130]:
q = Question(lc_client.getQuestionId(title), title)
lc_client.getSimilarQuestionCluster(q)

[5: https://leetcode.com/problems/longest-palindromic-substring/,
 28: https://leetcode.com/problems/implement-strstr/,
 31: https://leetcode.com/problems/next-permutation/,
 46: https://leetcode.com/problems/permutations/,
 47: https://leetcode.com/problems/permutations-ii/,
 49: https://leetcode.com/problems/group-anagrams/,
 60: https://leetcode.com/problems/permutation-sequence/,
 72: https://leetcode.com/problems/edit-distance/,
 214: https://leetcode.com/problems/shortest-palindrome/,
 242: https://leetcode.com/problems/valid-anagram/,
 249: https://leetcode.com/problems/group-shifted-strings/,
 266: https://leetcode.com/problems/palindrome-permutation/,
 267: https://leetcode.com/problems/palindrome-permutation-ii/,
 336: https://leetcode.com/problems/palindrome-pairs/,
 409: https://leetcode.com/problems/longest-palindrome/,
 438: https://leetcode.com/problems/find-all-anagrams-in-a-string/,
 459: https://leetcode.com/problems/repeated-substring-pattern/,
 516: https://leetcode

In [75]:
from time import sleep

questions = lc_client.getlAllQuestions()
cache = {q.titleSlug:q for q in questions}
visited = set()
for q in questions:
    if q.titleSlug not in visited:
        qcluster = lc_client.getSimilarQuestionCluster(q)
        for subq in qcluster:
            print(subq)
        print("")
        sleep(2)
        visited |= set(lc_client.toTitle(qcluster))

[1. two-sum]
[3. longest-substring-without-repeating-characters]
[5. longest-palindromic-substring]
[7. reverse-integer]
[8. string-to-integer-atoi]
[11. container-with-most-water]
[15. 3sum]
[16. 3sum-closest]
[17. letter-combinations-of-a-phone-number]
[18. 4sum]
[20. valid-parentheses]
[22. generate-parentheses]
[28. implement-strstr]
[30. substring-with-concatenation-of-all-words]
[31. next-permutation]
[32. longest-valid-parentheses]
[39. combination-sum]
[40. combination-sum-ii]
[42. trapping-rain-water]
[46. permutations]
[47. permutations-ii]
[49. group-anagrams]
[53. maximum-subarray]
[60. permutation-sequence]
[65. valid-number]
[72. edit-distance]
[76. minimum-window-substring]
[77. combinations]
[78. subsets]
[90. subsets-ii]
[94. binary-tree-inorder-traversal]
[95. unique-binary-search-trees-ii]
[96. unique-binary-search-trees]
[98. validate-binary-search-tree]
[102. binary-tree-level-order-traversal]
[103. binary-tree-zigzag-level-order-traversal]
[104. maximum-depth-of-b

HTTPError: 502 Server Error: Bad Gateway for url: https://leetcode.com/graphql